## 用机器学习方法完成中文文本分类
by 寒小阳(hanxiaoyang.ml@gmail.com)

### 朴素贝叶斯
我们试试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

机器学习的算法要取得好效果，离不开数据，咱们先把数据加载进来看看。

#### 准备数据
准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。

In [1]:
import jieba
import pandas as pd
df_technology = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [2]:
print (technology[12])
print (car[100])
print (entertainment[10])
print (military[10])
print (sports[10])

　　现在家里都拉了网线，都能无线上网，一定要帮他们先登上WiFi，另外，老人不懂得流量是什么，也不知道如何开关，控制流量，所以设置好流量上限很重要，免得不小心点开了视频或者下载，电话费就大发了。
　　截至发稿时，人人车给出的处理方案仍旧是检修车辆。王先生则认为，车辆在购买时就存在问题，但交易平台并未能检测出来。因此，王先生希望对方退款。王先生称，他将找专业机构对车辆进行鉴定，并通过法律途径维护自己的权益。J256
　　网综尺度相对较大原本是制作优势，《奇葩说》也曾经因为讨论的话题较为前卫一度引发争议。但《奇葩说》对于价值观的把握和引导让其中内含的“少儿不宜”只能算是小花絮。而纯粹是为了制造话题而“污”得“无节操无下限”的网综不仅让人生厌，也给节目自身招致了下架的厄运。对资本方而言，即便只从商业运营考量，点击量也分有价值和无价值，节目内容的变现能力如果建立在博眼球和低趣味迎合上，商业运营也难长久。对节目制作方与平台来说，为博一时的高点击而不顾底线不仅是砸自己招牌，以噱头吸引而来的观众与流量也是难以维持。
　　央视记者 胡善敏：我现在所处的位置是在辽宁舰的飞行甲板，执行跨海区训练和试验任务的辽宁舰官兵，正在展开多个科目的训练，穿着不同颜色服装的官兵在紧张的对舰载机进行转运。
　　据统计，2016年仅在中国田径协会注册的马拉松赛事便达到了328场，继续呈现出爆发式增长的态势，2015年，这个数字还仅仅停留在134场。如果算上未在中国田协注册的纯“民间”赛事，国内全年的路跑赛事还要更多。


### 分词与中文文本处理
#### 停用词

In [3]:
stopwords=pd.read_csv("/usr/local/codeData/jupyterData/jupyter-bd/textCategorization/data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [4]:
def preprocess_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = filter(lambda x:len(x)>1, segs)
            segs = filter(lambda x:x not in stopwords, segs)
            sentences.append((" ".join(segs), category))
        except Exception:
            print (line)
            continue 

#生成训练数据
sentences = []

preprocess_text(technology, sentences, 'technology')
preprocess_text(car, sentences, 'car')
preprocess_text(entertainment, sentences, 'entertainment')
preprocess_text(military, sentences, 'military')
preprocess_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.737 seconds.
Prefix dict has been built successfully.


#### 生成训练集

In [5]:
import random
random.shuffle(sentences)

for sentence in sentences[:10]:
    print (sentence[0], sentence[1])

中国 介绍 世界 entertainment
精准 用户 知晓 兴趣 购票 转化 口碑 转化成 IP 品牌价值 沉淀 二次 循环 entertainment
手动 休假 出征 military
被誉为 北疆 第一 伊木河 哨所 地处 大兴安岭 深处 冬季 奇寒 漫长 无霜期 多天 最低气温 零下 摄氏度 称为 中国 最冷 地方 military
广东 深圳 滨海 滨河 大道 启用 首条 乘员 车辆 专用车 载客 车辆 启用 时间段 工作日 早晚 高峰 car
竣工 仪式 尾声 基金会 企业 志愿者 孩子 文具 体育用品 郑海霞 同学 赠送 签名 篮球 勉励 孩子 好好学习 文化 知识 好好 锻炼身体 努力 祖国 栋梁 回报 社会 sports
2012 青海湖 景区 青海 市民 群众 推出 以爱为 主题 元旦 徒步 迎新年 活动 传达 低碳 环保 户外 旅游 理念 参与 人数 逐年 增加 公里 徒步 过程 捡拾 垃圾 公益活动 sports
胡先生 平时 上班 之余 微信 卖片 胡先生 来到 辖区 派出所 报警 银行卡 1818 不到 一分钟 买家 没收 民警 调查 发现 当日 时许 胡先生 接到 陌生人 发来 微信 说要 买部 电影 商议 价格 买家 扫描 二维码 付款 胡先生 没多想 微信 钱包 首页 二维码 条形码 截图 发给 买家 系统 延迟 截图 没收 胡先生 连发 三次 不到 一分钟 胡先生 收到 银行 告知 短信 银行卡 被扣 1818 胡先生 赶紧 买家 发现 微信 拉黑 technology
学习 强手 学习 强手 冰雪 运动 布局 我国 冰雪 运动 发展 偏科 现象 短道 速滑 花样滑冰 双人滑 冰壶 自由式 滑雪 空中 技巧 项目 奠定 优势 冰上 项目 项目 欠账 中国 冰雪 运动 想要 世界 占有 一席之地 改变 均衡 布局 项目 世界 内有 影响 项目 突破 冰雪 运动 布局 日本 韩国 哈萨克斯坦 值得 借鉴 sports
具备 下水 条件 military


为了一会儿检测一下咱们的分类器效果怎么样，我们需要一份测试集。

所以把原数据集分成训练集的测试集，咱们用sklearn自带的分割函数。

In [6]:
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

In [7]:
len(x_train)

65696

下一步要做的就是在降噪数据上抽取出来有用的特征啦，我们对文本抽取词袋模型特征

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    max_features=4000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [9]:
vec

CountVectorizer(max_features=4000)

把分类器import进来并且训练

In [10]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB()

看看我们的准确率如何

In [11]:
classifier.score(vec.transform(x_test), y_test)

0.8404036714005205

In [12]:
len(x_test)

21899

我们可以看到在2w多个样本上，我们能在5个类别上达到83的准确率。

有没有办法把准确率提高一些呢？

我们可以把特征做得更棒一点，比如说，我们试试加入抽取2-gram和3-gram的统计特征，比如可以把词库的量放大一点。

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,4),  # use ngrams of size 1 and 2
    max_features=20000,  # keep the most common 1000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

#### 分类训练

In [14]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.8789442440294077

#### 交叉验证

更可靠的验证效果的方式是交叉验证，但是交叉验证最好保证每一份里面的样本类别也是相对均衡的，我们这里使用StratifiedKFold

In [15]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np

def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    #stratifiedk_fold = StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    stratifiedk_fold = StratifiedKFold(n_splits=5)
    y_pred = y[:]
    for train_index, test_index in stratifiedk_fold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred 

NB = MultinomialNB
print (precision_score(y, stratifiedkfold_cv(vec.transform(x),np.array(y),NB), average='macro'))

0.8805366237978403


我们做完K折的交叉验证，可以看到在5个类别上的结果平均准确度约为88%

### 我们自己来完成一个文本分类器class

In [16]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)


In [17]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8789442440294077


### SVM文本分类
我们来试试支持向量机的作用

In [18]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8494908443307914

可以试试rbf核

In [19]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8488515457326818

### 换特征/模型试试

In [20]:
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [21]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.877893967761085
